# 3: Test case: changing the emissions


<div class="alert alert-info">
<strong>Exercise 3: Using different emissions</strong><br><br>
 
Create, configure, build a new case as done in exercise 1 or 2, for one of these following configurations: 
- **CAM-chem:** ``f.e21.FCHIST.f09_f09_mg17.tutorial.test3`` with the ``FCHIST`` compset; or 
- **WACCM:** ``f.e21.FWHIST.f09_f09_mg17.tutorial.test3`` with the ``FWHIST`` compset. 

Set up this simulation to run for 5 days, with daily instantaneous output of the variables: ‘PS’,’Z3’,’T’,’U’,’V’,’O3’. 
    
Change the namelist variables to use daily fire emissions from QFED. This will also require a change of model STARTDATE. Once the simulation is complete, compare the 5th day output to the previous results.
    
    
</div>



<div class="alert alert-warning">  
<details>

<summary> <font face="Times New Roman" color='blue'>Click here for hints</font> </summary>
<br>

**Pointing to new emission files**

Point to new emission files in your ``user_nl_cam`` file. Note that you will also need to include pointers to emissions that you are not changing. The namelist variable for emissions is srf_emis_specifier. Copy the whole srf_emis_specifier entry from ``atm_in`` to ``user_nl_cam`` before editing. Emission files will add up in CESM.

The location of qfed fire emissions on glade is described on the wiki: https://wiki.ucar.edu/display/camchem/Emission+Inventories
Search for files with bb in their name, which are fire / biomass burning emissions. Global one degree emissions are usually stored under an 'f09' folder.
    
**Make sure your model start date is covered by the emissions files**
For this model set up will also need to double check the following are available:
- The SST file covers your selected dates (``env_run.xml`` file)
- Lower boundary condition file must include your selected dates (``atm_in`` file)
- Consider that the initialization file will be for the default 1995. If you don't change your initialization file, you will need to run your own model spin up.


</details>
</div>



<div class="alert alert-success">   
<details>
<summary><font face="Times New Roman" color='blue'>Click here for the solution</font></summary><br>
 
 
**# Set environment variables**
Use the commands:
   
**CAM-chem**
```
set CASENAME = f.e21.FCHIST.f09_f09_mg17.tutorial.test3
set CASEDIR = /glade/u/home/$USER/cases/$CASENAME
set RUNDIR = /glade/derecho/scratch/$USER/$CASENAME/run
set COMPSET = FCHIST
set RESOLUTION = f09_f09_mg17
```

or for **WACCM**
    
```
set CASENAME = f.e21.FWHIST.f09_f09_mg17.tutorial.test3
set CASEDIR = /glade/u/home/$USER/cases/$CASENAME
set RUNDIR = /glade/derecho/scratch/$USER/$CASENAME/run
set COMPSET = FWHIST
set RESOLUTION = f09_f09_mg17
```

**# Create a new case**

Create a new case with the command ``create_newcase``:
```
cd /glade/u/home/$USER/code/my_cesm_code/cime/scripts/
./create_newcase --case $CASEDIR --res $RESOLUTION --compset $COMPSET
```
You may need to add ``--project "UESM0013"`` when creating a new case.

**# Change the job queue and account number**

**If needed**, change ``job queue`` and ``account number``.<br>
For instance, to run in the queue ``tutorial`` and the project number ``UESM0013``
```  
cd $CASEDIR
./xmlchange JOB_QUEUE=tutorial,PROJECT=UESM0013 --force
```
This step can be redone at anytime in the process. 

**# Setup**

From within the case directory, invoke ``case.setup`` with the command:
```    
cd $CASEDIR
./case.setup    
```  
    
**# Change the dates to cover emission files** <br>
Here, we choose 2010
```
./xmlchange RUN_STARTDATE=2010-01-01
```

Make sure the sea surface temperature and ice (SSTICE) data file covers your dates of interest
```
./xmlquery SSTICE_DATA_FILENAME
```
    
should return: 
```
SSTICE_DATA_FILENAME: /glade/campaign/cesm/cesmdata/cseg/inputdata/atm/cam/sst/sst_HadOIBl_bc_0.9x1.25_1850_2017_c180507.nc
```

    
**# Build**
    
Perform an initial build within the case directory
```
qcmd -- ./case.build
```
namelists (``atm_in``, ``ice_in``, ``lnd_in``, ``docn_in``) will appear in the CaseDocs
subdirectory, as well as in your $rundir

**# Customize namelists**

As before, edit the file ``user_nl_cam``. First open the ``CaseDocs/atm_in`` and find the lines for avgflag_pertap, mfilt, nhtfrq and copy and paste them into ``user_nl_cam``. Then make changes to those lines in ``user_nl_cam`` and add a line to write out a second history file, ``fincl2``. The namelist lines should look something like:
```
&cam_history_nl
 avgflag_pertape = 'A','I','A','A','A','A','A','A','I'
 mfilt          = 1,30,365,240,240,480,365,73,30
 nhtfrq         = 0,-24,-24,-3,-1,1,-24,-120,-240
 fincl2         = 'PS','Z3','T','U','V','O3'
/
``` 
You can do this with a text editor.

**# Add in different emissions**
Also add in the namelist pointers to emission files, replacing the fire emissions (every file with 'bb' in the name) with qfed files found at ``/glade/campaign/acom/MUSICA/emis/qfed/qfed2.5_finn/f09/``

```
&chem_inparm
 srf_emis_specifier		= 'BENZENE -> /glade/campaign/cesm/cesmdata/cseg/inputdata/atm/cam/chem/emis/CMIP6_emissions_1750_2015/emissions-cmip6_BENZENE_anthro_surface_1750-2015_0.9x1.25_c20170608.nc',
         'BENZENE -> /glade/campaign/acom/MUSICA/emis/qfed/qfed2.5_finn/f09/qfed.emis_BENZENE_0.9x1.25_mol_2000_2022.nc',
         'BIGALK -> /glade/campaign/cesm/cesmdata/cseg/inputdata/atm/cam/chem/emis/CMIP6_emissions_1750_2015/emissions-cmip6_BIGALK_anthro_surface_1750-2015_0.9x1.25_c20170608.nc',
         'BIGALK -> /glade/campaign/acom/MUSICA/emis/qfed/qfed2.5_finn/f09/qfed.emis_BIGALK_0.9x1.25_mol_2000_2022.nc',
         'BIGENE -> /glade/campaign/cesm/cesmdata/cseg/inputdata/atm/cam/chem/emis/CMIP6_emissions_1750_2015/emissions-cmip6_BIGENE_anthro_surface_1750-2015_0.9x1.25_c20170608.nc',
         'BIGENE -> /glade/campaign/acom/MUSICA/emis/qfed/qfed2.5_finn/f09/qfed.emis_BIGENE_0.9x1.25_mol_2000_2022.nc',
         'C2H2 -> /glade/campaign/cesm/cesmdata/cseg/inputdata/atm/cam/chem/emis/CMIP6_emissions_1750_2015/emissions-cmip6_C2H2_anthro_surface_1750-2015_0.9x1.25_c20170608.nc',
         'C2H2 -> /glade/campaign/acom/MUSICA/emis/qfed/qfed2.5_finn/f09/qfed.emis_C2H2_0.9x1.25_mol_2000_2022.nc',
         'C2H4 -> /glade/campaign/cesm/cesmdata/cseg/inputdata/atm/cam/chem/emis/CMIP6_emissions_1750_2015/emissions-cmip6_C2H4_anthro_surface_1750-2015_0.9x1.25_c20170608.nc',
         'C2H4 -> /glade/campaign/acom/MUSICA/emis/qfed/qfed2.5_finn/f09/qfed.emis_C2H4_0.9x1.25_mol_2000_2022.nc',
         'C2H4 -> /glade/campaign/cesm/cesmdata/cseg/inputdata/atm/cam/chem/emis/CMIP6_emissions_1750_2015/emissions-cmip6_C2H4_other_surface_1750-2015_0.9x1.25_c20170322.nc',
         'C2H5OH -> /glade/campaign/cesm/cesmdata/cseg/inputdata/atm/cam/chem/emis/CMIP6_emissions_1750_2015/emissions-cmip6_C2H5OH_anthro_surface_1750-2015_0.9x1.25_c20170608.nc',
         'C2H5OH -> /glade/campaign/acom/MUSICA/emis/qfed/qfed2.5_finn/f09/qfed.emis_C2H5OH_0.9x1.25_mol_2000_2022.nc',
         'C2H6 -> /glade/campaign/cesm/cesmdata/cseg/inputdata/atm/cam/chem/emis/CMIP6_emissions_1750_2015/emissions-cmip6_C2H6_anthro_surface_1750-2015_0.9x1.25_c20170608.nc',
         'C2H6 -> /glade/campaign/acom/MUSICA/emis/qfed/qfed2.5_finn/f09/qfed.emis_C2H6_0.9x1.25_mol_2000_2022.nc',
         'C2H6 -> /glade/campaign/cesm/cesmdata/cseg/inputdata/atm/cam/chem/emis/CMIP6_emissions_1750_2015/emissions-cmip6_C2H6_other_surface_1750-2015_0.9x1.25_c20170322.nc',
         'C3H6 -> /glade/campaign/cesm/cesmdata/cseg/inputdata/atm/cam/chem/emis/CMIP6_emissions_1750_2015/emissions-cmip6_C3H6_anthro_surface_1750-2015_0.9x1.25_c20170608.nc',
         'C3H6 -> /glade/campaign/acom/MUSICA/emis/qfed/qfed2.5_finn/f09/qfed.emis_C3H6_0.9x1.25_mol_2000_2022.nc',
         'C3H6 -> /glade/campaign/cesm/cesmdata/cseg/inputdata/atm/cam/chem/emis/CMIP6_emissions_1750_2015/emissions-cmip6_C3H6_other_surface_1750-2015_0.9x1.25_c20170322.nc',
         'C3H8 -> /glade/campaign/cesm/cesmdata/cseg/inputdata/atm/cam/chem/emis/CMIP6_emissions_1750_2015/emissions-cmip6_C3H8_anthro_surface_1750-2015_0.9x1.25_c20170608.nc',
         'C3H8 -> /glade/campaign/acom/MUSICA/emis/qfed/qfed2.5_finn/f09/qfed.emis_C3H8_0.9x1.25_mol_2000_2022.nc',
         'C3H8 -> /glade/campaign/cesm/cesmdata/cseg/inputdata/atm/cam/chem/emis/CMIP6_emissions_1750_2015/emissions-cmip6_C3H8_other_surface_1750-2015_0.9x1.25_c20170322.nc',
         'CH2O -> /glade/campaign/cesm/cesmdata/cseg/inputdata/atm/cam/chem/emis/CMIP6_emissions_1750_2015/emissions-cmip6_CH2O_anthro_surface_1750-2015_0.9x1.25_c20170608.nc',
         'CH2O -> /glade/campaign/acom/MUSICA/emis/qfed/qfed2.5_finn/f09/qfed.emis_CH2O_0.9x1.25_mol_2000_2022.nc',
         'CH3CHO -> /glade/campaign/cesm/cesmdata/cseg/inputdata/atm/cam/chem/emis/CMIP6_emissions_1750_2015/emissions-cmip6_CH3CHO_anthro_surface_1750-2015_0.9x1.25_c20170608.nc',
         'CH3CHO -> /glade/campaign/acom/MUSICA/emis/qfed/qfed2.5_finn/f09/qfed.emis_CH3CHO_0.9x1.25_mol_2000_2022.nc',
         'CH3CN -> /glade/campaign/cesm/cesmdata/cseg/inputdata/atm/cam/chem/emis/CMIP6_emissions_1750_2015/emissions-cmip6_CH3CN_anthro_surface_1750-2015_0.9x1.25_c20170608.nc',
         'CH3CN -> /glade/campaign/acom/MUSICA/emis/qfed/qfed2.5_finn/f09/qfed.emis_CH3CN_0.9x1.25_mol_2000_2022.nc',
         'CH3COCH3 -> /glade/campaign/cesm/cesmdata/cseg/inputdata/atm/cam/chem/emis/CMIP6_emissions_1750_2015/emissions-cmip6_CH3COCH3_anthro_surface_1750-2015_0.9x1.25_c20170608.nc',
         'CH3COCH3 -> /glade/campaign/acom/MUSICA/emis/qfed/qfed2.5_finn/f09/qfed.emis_CH3COCH3_0.9x1.25_mol_2000_2022.nc',
         'CH3COCHO -> /glade/campaign/acom/MUSICA/emis/qfed/qfed2.5_finn/f09/qfed.emis_CH3COCHO_0.9x1.25_mol_2000_2022.nc',
         'CH3COOH -> /glade/campaign/cesm/cesmdata/cseg/inputdata/atm/cam/chem/emis/CMIP6_emissions_1750_2015/emissions-cmip6_CH3COOH_anthro_surface_1750-2015_0.9x1.25_c20170608.nc',
         'CH3COOH -> /glade/campaign/acom/MUSICA/emis/qfed/qfed2.5_finn/f09/qfed.emis_CH3COOH_0.9x1.25_mol_2000_2022.nc',
         'CH3OH -> /glade/campaign/cesm/cesmdata/cseg/inputdata/atm/cam/chem/emis/CMIP6_emissions_1750_2015/emissions-cmip6_CH3OH_anthro_surface_1750-2015_0.9x1.25_c20170608.nc',
         'CH3OH -> /glade/campaign/acom/MUSICA/emis/qfed/qfed2.5_finn/f09/qfed.emis_CH3OH_0.9x1.25_mol_2000_2022.nc',
         'CO -> /glade/campaign/cesm/cesmdata/cseg/inputdata/atm/cam/chem/emis/CMIP6_emissions_1750_2015/emissions-cmip6_CO_anthro_surface_1750-2015_0.9x1.25_c20180504.nc',
         'CO -> /glade/campaign/acom/MUSICA/emis/qfed/qfed2.5_finn/f09/qfed.emis_CO_0.9x1.25_mol_2000_2022.nc',
         'CO -> /glade/campaign/cesm/cesmdata/cseg/inputdata/atm/cam/chem/emis/CMIP6_emissions_1750_2015/emissions-cmip6_CO_other_surface_1750-2015_0.9x1.25_c20170322.nc',
         'E90 -> /glade/campaign/cesm/cesmdata/cseg/inputdata/atm/cam/chem/emis/CMIP6_emissions_1750_2015/emissions_E90global_surface_1750-2100_0.9x1.25_c20170322.nc',
         'GLYALD -> /glade/campaign/acom/MUSICA/emis/qfed/qfed2.5_finn/f09/qfed.emis_GLYALD_0.9x1.25_mol_2000_2022.nc',
         'HCN -> /glade/campaign/cesm/cesmdata/cseg/inputdata/atm/cam/chem/emis/CMIP6_emissions_1750_2015/emissions-cmip6_HCN_anthro_surface_1750-2015_0.9x1.25_c20170608.nc',
         'HCN -> /glade/campaign/acom/MUSICA/emis/qfed/qfed2.5_finn/f09/qfed.emis_HCN_0.9x1.25_mol_2000_2022.nc',
         'HCOOH -> /glade/campaign/cesm/cesmdata/cseg/inputdata/atm/cam/chem/emis/CMIP6_emissions_1750_2015/emissions-cmip6_HCOOH_anthro_surface_1750-2015_0.9x1.25_c20170608.nc',
         'HCOOH -> /glade/campaign/acom/MUSICA/emis/qfed/qfed2.5_finn/f09/qfed.emis_HCOOH_0.9x1.25_mol_2000_2022.nc',
         'ISOP -> /glade/campaign/acom/MUSICA/emis/qfed/qfed2.5_finn/f09/qfed.emis_ISOP_0.9x1.25_mol_2000_2022.nc',
         'IVOC -> /glade/campaign/cesm/cesmdata/cseg/inputdata/atm/cam/chem/emis/CMIP6_emissions_1750_2015/emissions-cmip6_IVOC_anthro_surface_1750-2015_0.9x1.25_c20170608.nc',
         'IVOC -> /glade/campaign/acom/MUSICA/emis/qfed/qfed2.5_finn/f09/qfed.emis_IVOC_0.9x1.25_mol_2000_2022.nc',
         'MEK -> /glade/campaign/cesm/cesmdata/cseg/inputdata/atm/cam/chem/emis/CMIP6_emissions_1750_2015/emissions-cmip6_MEK_anthro_surface_1750-2015_0.9x1.25_c20170608.nc',
         'MEK -> /glade/campaign/acom/MUSICA/emis/qfed/qfed2.5_finn/f09/qfed.emis_MEK_0.9x1.25_mol_2000_2022.nc',
         'MTERP -> /glade/campaign/acom/MUSICA/emis/qfed/qfed2.5_finn/f09/qfed.emis_TERPENES_0.9x1.25_mol_2000_2022.nc ',
         'NH3 -> /glade/campaign/cesm/cesmdata/cseg/inputdata/atm/cam/chem/emis/CMIP6_emissions_1750_2015/emissions-cmip6_NH3_anthro_surface_1750-2015_0.9x1.25_c20170608.nc',
         'NH3 -> /glade/campaign/acom/MUSICA/emis/qfed/qfed2.5_finn/f09/qfed.emis_NH3_0.9x1.25_mol_2000_2022.nc ',
         'NH3 -> /glade/campaign/cesm/cesmdata/cseg/inputdata/atm/cam/chem/emis/CMIP6_emissions_1750_2015/emissions-cmip6_NH3_other_surface_1750-2015_0.9x1.25_c20170322.nc',
         'NO -> /glade/campaign/cesm/cesmdata/cseg/inputdata/atm/cam/chem/emis/CMIP6_emissions_1750_2015/emissions-cmip6_NO_anthro_surface_1750-2015_0.9x1.25_c20170608.nc',
         'NO -> /glade/campaign/acom/MUSICA/emis/qfed/qfed2.5_finn/f09/qfed.emis_NO_0.9x1.25_mol_2000_2022.nc ',
         'NO -> /glade/campaign/cesm/cesmdata/cseg/inputdata/atm/cam/chem/emis/CMIP6_emissions_1750_2015/emissions-cmip6_NO_other_surface_1750-2015_0.9x1.25_c20170322.nc',
         'SVOC -> /glade/campaign/cesm/cesmdata/cseg/inputdata/atm/cam/chem/emis/CMIP6_emissions_1750_2015/emissions-cmip6_SVOC_anthro_surface_1750-2015_0.9x1.25_c20170608.nc',
         'SVOC -> /glade/campaign/acom/MUSICA/emis/qfed/qfed2.5_finn/f09/qfed.emis_SVOC_0.9x1.25_mol_2000_2022.nc ',
         'TOLUENE -> /glade/campaign/cesm/cesmdata/cseg/inputdata/atm/cam/chem/emis/CMIP6_emissions_1750_2015/emissions-cmip6_TOLUENE_anthro_surface_1750-2015_0.9x1.25_c20170608.nc',
         'TOLUENE -> /glade/campaign/acom/MUSICA/emis/qfed/qfed2.5_finn/f09/qfed.emis_TOLUENE_0.9x1.25_mol_2000_2022.nc ',
         'XYLENES -> /glade/campaign/cesm/cesmdata/cseg/inputdata/atm/cam/chem/emis/CMIP6_emissions_1750_2015/emissions-cmip6_XYLENES_anthro_surface_1750-2015_0.9x1.25_c20170608.nc',
         'XYLENES -> /glade/campaign/acom/MUSICA/emis/qfed/qfed2.5_finn/f09/qfed.emis_XYLENES_0.9x1.25_mol_2000_2022.nc ',
         'bc_a4 -> /glade/campaign/cesm/cesmdata/cseg/inputdata/atm/cam/chem/emis/CMIP6_emissions_1750_2015/emissions-cmip6_bc_a4_anthro_surface_1750-2015_0.9x1.25_c20170608.nc',
         'bc_a4 -> /glade/campaign/acom/MUSICA/emis/qfed/qfed2.5_finn/f09/qfed.emis_bc_a4_0.9x1.25_mol_2000_2022.nc ',
         'DMS -> /glade/campaign/acom/MUSICA/emis/qfed/qfed2.5_finn/f09/qfed.emis_DMS_0.9x1.25_mol_2000_2022.nc ',
         'DMS -> /glade/campaign/cesm/cesmdata/cseg/inputdata/atm/cam/chem/emis/CMIP6_emissions_1750_2015/emissions-cmip6_DMS_other_surface_1750-2015_0.9x1.25_c20170322.nc',
         'num_a1 -> /glade/campaign/acom/MUSICA/emis/qfed/qfed2.5_finn/f09/qfed.emis_num_so4_a1_0.9x1.25_mol_2000_2022.nc ',
         'num_a1 -> /glade/campaign/cesm/cesmdata/cseg/inputdata/atm/cam/chem/emis/CMIP6_emissions_1750_2015/emissions-cmip6_num_so4_a1_anthro-ag-ship_surface_1750-2015_0.9x1.25_c20170616.nc',
         'num_a2 -> /glade/campaign/cesm/cesmdata/cseg/inputdata/atm/cam/chem/emis/CMIP6_emissions_1750_2015/emissions-cmip6_num_so4_a2_anthro-res_surface_1750-2015_0.9x1.25_c20170616.nc',
         'num_a4 -> /glade/campaign/acom/MUSICA/emis/qfed/qfed2.5_finn/f09/qfed.emis_num_bc_a4_0.9x1.25_mol_2000_2022.nc ',
         'num_a4 -> /glade/campaign/cesm/cesmdata/cseg/inputdata/atm/cam/chem/emis/CMIP6_emissions_1750_2015/emissions-cmip6_num_bc_a4_anthro_surface_1750-2015_0.9x1.25_c20170608.nc',
         'num_a4 -> /glade/campaign/cesm/cesmdata/cseg/inputdata/atm/cam/chem/emis/CMIP6_emissions_1750_2015/emissions-cmip6_num_pom_a4_anthro_surface_1750-2015_0.9x1.25_c20170608.nc',
         'num_a4 -> /glade/campaign/acom/MUSICA/emis/qfed/qfed2.5_finn/f09/qfed.emis_num_pom_a4_0.9x1.25_mol_2000_2022.nc ',
         'pom_a4 -> /glade/campaign/cesm/cesmdata/cseg/inputdata/atm/cam/chem/emis/CMIP6_emissions_1750_2015/emissions-cmip6_pom_a4_anthro_surface_1750-2015_0.9x1.25_c20170608.nc',
         'pom_a4 -> /glade/campaign/acom/MUSICA/emis/qfed/qfed2.5_finn/f09/qfed.emis_pom_a4_0.9x1.25_mol_2000_2022.nc ',
         'SO2 -> /glade/campaign/cesm/cesmdata/cseg/inputdata/atm/cam/chem/emis/CMIP6_emissions_1750_2015/emissions-cmip6_SO2_anthro-ag-ship-res_surface_1750-2015_0.9x1.25_c20170616.nc',
         'SO2 -> /glade/campaign/cesm/cesmdata/cseg/inputdata/atm/cam/chem/emis/CMIP6_emissions_1750_2015/emissions-cmip6_SO2_anthro-ene_surface_1750-2015_0.9x1.25_c20170616.nc',
         'SO2 -> /glade/campaign/acom/MUSICA/emis/qfed/qfed2.5_finn/f09/qfed.emis_SO2_0.9x1.25_mol_2000_2022.nc',
         'so4_a1 -> /glade/campaign/cesm/cesmdata/cseg/inputdata/atm/cam/chem/emis/CMIP6_emissions_1750_2015/emissions-cmip6_so4_a1_anthro-ag-ship_surface_1750-2015_0.9x1.25_c20170616.nc',
         'so4_a1 -> /glade/campaign/acom/MUSICA/emis/qfed/qfed2.5_finn/f09/qfed.emis_so4_a1_0.9x1.25_mol_2000_2022.nc ',
         'so4_a2 -> /glade/campaign/cesm/cesmdata/cseg/inputdata/atm/cam/chem/emis/CMIP6_emissions_1750_2015/emissions-cmip6_so4_a2_anthro-res_surface_1750-2015_0.9x1.25_c20170616.nc'
/
```

**# Lower boundary condition file check**

Make sure the LBC files cover your dates of interest. In ``CaseDocs/atm_in`` you should see:
```
 flbc_file              = '/glade/campaign/cesm/cesmdata/cseg/inputdata/atm/waccm/lb/LBC_1750-2014_CMIP6_0p5degLat_c170126.nc'
```

**# Initialization files for your new startdate**

Note: Below are initialization files for 2010. For scientifically valid simulations you need to point to an initialization file for the start date you choose. If one is not available, you will need to spin up the model.

For **CAM-chem**:
atmospheric initialization files are redefined in ``user_nl_cam``

```
&cam_initfiles_nl
 ncdata         = '/glade/campaign/cesm/acom/acom-climate/tilmes/inputdata/init/camchem/FCnudged_MAM4_f09.carma_trop_strat09.aqchem.2001_2020.cam.i.2010-01-01-00000.nc'
/
```


For **WACCM**:
atmospheric initialization files are redefined in ``user_nl_cam``

```
&cam_initfiles_nl
 ncdata         = '/glade/campaign/cesm/acom/acom-climate/tilmes/inputdata/init/waccm/f.e21.FWHISTBgcCrop.f09_f09_mg17.CMIP6-AMIP-WACCM.001.cam.i.2010-01-01-00000.nc'
/
```    


You build the namelists with the command:
```
./preview_namelists
```
This step is optional as the script ``preview_namelists`` is automatically called by ``case.build`` and ``case.submit``. But it is nice to check that your changes made their way into:
```
$CASEDIR/CaseDocs/atm_in
```


**# Set run length**

If needed, change the ``run length``. If you want to run 5 days, you don't have to do this, as 5 days is the default. 
```   
./xmlchange STOP_N=5,STOP_OPTION=ndays
```


**# Submit:**
```
./case.submit
```
------------



**# Check your solution**

When the run is completed, look at the run files and history files.  
    
(1)  Find your model output in your run directory (``$run_dir``) after finished:

```
ls /glade/derecho/scratch/$USER/$CASENAME/run
```
    
(2) Check that your archive directory on derecho (the path will be different on other machines): 
```
cd /glade/derecho/scratch/$USER/archive/$CASENAME/atm/hist
ls 
```

As your run is only 5-day, there should be no monthly file (``h0``)

(3) Look at the contents of the ``h1`` files using ``ncdump``.

```
ncdump –h f.e21.FCHIST.f09_f09_mg17.tutorial.test3.cam.h1.2010-01-01-00000.nc
```

</details>
</div>

